In [1]:
import pandas as pd
import numpy as np

In [45]:
#load the representative data and limit it to only 2010 and above
res = pd.read_csv("1976-2018-house3.csv")
res_12 = res.loc[res["year"] >= 2010]

Filtering the winners

In [46]:
#Since we are given all candidates with their votes, we must group by then take the person with the max votes in each group.
select = res_12.groupby(["year","state_fips", "district"])["candidatevotes"].transform(max) == res_12["candidatevotes"]

In [47]:
winner = res_12[select]

In [48]:
winner

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
22536,2010,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,JO BONNER,REPUBLICAN,False,TOTAL,129063,156281,False,20201202,False
22541,2010,ALABAMA,AL,1,63,41,US HOUSE,2,gen,NaN,False,MARTHA ROBY,REPUBLICAN,False,TOTAL,111645,219028,False,20201202,False
22542,2010,ALABAMA,AL,1,63,41,US HOUSE,3,gen,NaN,False,MIKE ROGERS,REPUBLICAN,False,TOTAL,117736,198139,False,20201202,False
22545,2010,ALABAMA,AL,1,63,41,US HOUSE,4,gen,NaN,False,ROBERT B. ADERHOLT,REPUBLICAN,False,TOTAL,167714,169721,False,20201202,False
22549,2010,ALABAMA,AL,1,63,41,US HOUSE,5,gen,NaN,False,MO BROOKS,REPUBLICAN,False,TOTAL,131109,226490,False,20201202,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,2018,MARYLAND,MD,24,52,52,US HOUSE,4,gen,NaN,False,ANTHONY G. BROWN,DEMOCRAT,False,TOTAL,209642,268583,False,20201202,False
29620,2018,MARYLAND,MD,24,52,52,US HOUSE,5,gen,NaN,False,STENY H. HOYER,DEMOCRAT,False,TOTAL,213796,304209,False,20201202,False
29624,2018,MARYLAND,MD,24,52,52,US HOUSE,6,gen,NaN,False,DAVID TRONE,DEMOCRAT,False,TOTAL,163346,276974,False,20201202,False
29629,2018,MARYLAND,MD,24,52,52,US HOUSE,7,gen,NaN,False,ELIJAH CUMMINGS,DEMOCRAT,False,TOTAL,202345,264710,False,20201202,False


Adding flipped column

In [49]:
groups = winner.groupby(["state_fips", "district"], as_index=False)

In [50]:
winner.loc[(winner["state_fips"] == 1) & (winner["district"] ==1)]

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
22536,2010,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,JO BONNER,REPUBLICAN,False,TOTAL,129063,156281,False,20201202,False
24037,2012,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,JO BONNER,REPUBLICAN,False,TOTAL,196374,200676,False,20201202,False
25459,2014,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,BRADLEY BYRNE,REPUBLICAN,False,TOTAL,103758,152234,False,20201202,False
26816,2016,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,BRADLEY BYRNE,REPUBLICAN,False,TOTAL,208083,215893,False,20201202,False
28252,2018,ALABAMA,AL,1,63,41,US HOUSE,1,gen,NaN,False,BRADLEY BYRNE,REPUBLICAN,False,TOTAL,153228,242617,False,20201202,False


In [51]:
states = winner["state_fips"].unique()
winner.insert(0,"flipped", np.nan)

In [ ]:
# For all states
for i in states:
    districts = winner.loc[winner["state_fips"] == i]["district"].unique()
    # For all distritcs of each state
    for j in districts:
        district = winner.loc[(winner["state_fips"] == i) & (winner["district"] == j)]
        district = district.sort_values("year")
        years = district["year"].tolist()
        #for each specific year in each district,
        for k in range(1,len(years)):
            #compare previous years party with current years party. True if not equal (aka flipped)
            comp = winner.loc[(winner["state_fips"] == i) & (winner["district"] == j) & (winner["year"] == years[k])]["party"].values != \
            winner.loc[(winner["state_fips"] == i) & (winner["district"] == j) & (winner["year"] == years[k-1])]["party"].values
            #put boolean in the dataframe 
            ind = winner.loc[(winner["state_fips"] == i) & (winner["district"] == j) & (winner["year"] == years[k])].index
            winner.at[ind,"flipped"] = comp
                              
            
            
            
    

In [53]:
years = district["year"].tolist()
years

[2010, 2012, 2014, 2016, 2018]

In [54]:
winner

,flipped,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,...,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
22536,NaN,2010,ALABAMA,AL,1,63,41,US HOUSE,1,gen,...,False,JO BONNER,REPUBLICAN,False,TOTAL,129063,156281,False,20201202,False
22541,NaN,2010,ALABAMA,AL,1,63,41,US HOUSE,2,gen,...,False,MARTHA ROBY,REPUBLICAN,False,TOTAL,111645,219028,False,20201202,False
22542,NaN,2010,ALABAMA,AL,1,63,41,US HOUSE,3,gen,...,False,MIKE ROGERS,REPUBLICAN,False,TOTAL,117736,198139,False,20201202,False
22545,NaN,2010,ALABAMA,AL,1,63,41,US HOUSE,4,gen,...,False,ROBERT B. ADERHOLT,REPUBLICAN,False,TOTAL,167714,169721,False,20201202,False
22549,NaN,2010,ALABAMA,AL,1,63,41,US HOUSE,5,gen,...,False,MO BROOKS,REPUBLICAN,False,TOTAL,131109,226490,False,20201202,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,False,2018,MARYLAND,MD,24,52,52,US HOUSE,4,gen,...,False,ANTHONY G. BROWN,DEMOCRAT,False,TOTAL,209642,268583,False,20201202,False
29620,False,2018,MARYLAND,MD,24,52,52,US HOUSE,5,gen,...,False,STENY H. HOYER,DEMOCRAT,False,TOTAL,213796,304209,False,20201202,False
29624,False,2018,MARYLAND,MD,24,52,52,US HOUSE,6,gen,...,False,DAVID TRONE,DEMOCRAT,False,TOTAL,163346,276974,False,20201202,False
29629,False,2018,MARYLAND,MD,24,52,52,US HOUSE,7,gen,...,False,ELIJAH CUMMINGS,DEMOCRAT,False,TOTAL,202345,264710,False,20201202,False


In [55]:
end = winner.dropna(subset=["flipped"])

In [56]:
end

,flipped,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,...,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
24037,False,2012,ALABAMA,AL,1,63,41,US HOUSE,1,gen,...,False,JO BONNER,REPUBLICAN,False,TOTAL,196374,200676,False,20201202,False
24040,False,2012,ALABAMA,AL,1,63,41,US HOUSE,2,gen,...,False,MARTHA ROBY,REPUBLICAN,False,TOTAL,180591,283953,False,20201202,False
24041,False,2012,ALABAMA,AL,1,63,41,US HOUSE,3,gen,...,False,MIKE ROGERS,REPUBLICAN,False,TOTAL,175306,273930,False,20201202,False
24046,False,2012,ALABAMA,AL,1,63,41,US HOUSE,4,gen,...,False,ROBERT B. ADERHOLT,REPUBLICAN,False,TOTAL,199071,269118,False,20201202,False
24049,False,2012,ALABAMA,AL,1,63,41,US HOUSE,5,gen,...,False,MO BROOKS,REPUBLICAN,False,TOTAL,189185,291293,False,20201202,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29614,False,2018,MARYLAND,MD,24,52,52,US HOUSE,4,gen,...,False,ANTHONY G. BROWN,DEMOCRAT,False,TOTAL,209642,268583,False,20201202,False
29620,False,2018,MARYLAND,MD,24,52,52,US HOUSE,5,gen,...,False,STENY H. HOYER,DEMOCRAT,False,TOTAL,213796,304209,False,20201202,False
29624,False,2018,MARYLAND,MD,24,52,52,US HOUSE,6,gen,...,False,DAVID TRONE,DEMOCRAT,False,TOTAL,163346,276974,False,20201202,False
29629,False,2018,MARYLAND,MD,24,52,52,US HOUSE,7,gen,...,False,ELIJAH CUMMINGS,DEMOCRAT,False,TOTAL,202345,264710,False,20201202,False


In [57]:
end.loc[(end["state_po"] == "VA") & (end["district"] == 2)]

,flipped,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,...,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
25359,False,2012,VIRGINIA,VA,51,54,40,US HOUSE,2,gen,...,False,E. SCOTT RIGELL,REPUBLICAN,False,TOTAL,166231,309222,False,20201202,False
26706,False,2014,VIRGINIA,VA,51,54,40,US HOUSE,2,gen,...,False,E. SCOTT RIGELL,REPUBLICAN,False,TOTAL,101558,173060,False,20201202,False
28155,False,2016,VIRGINIA,VA,51,54,40,US HOUSE,2,gen,...,False,SCOTT W. TAYLOR,REPUBLICAN,False,TOTAL,190475,310567,False,20201202,False
29471,True,2018,VIRGINIA,VA,51,54,40,US HOUSE,2,gen,...,False,ELAINE G. LURIA,DEMOCRAT,False,TOTAL,139571,273400,False,20201202,False


In [58]:
end["year"].unique()

array([2012, 2014, 2016, 2018], dtype=int64)

In [59]:
end.to_csv("reps.csv", index = False)

In [44]:
end.loc[end["flipped"] == True]

,flipped,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,...,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
25486,True,2014,ARIZONA,AZ,4,86,61,US HOUSE,2,gen,...,False,MARTHA MCSALLY,REPUBLICAN,False,TOTAL,109704,219351,False,20201202,False
25585,True,2014,CALIFORNIA,CA,6,93,71,US HOUSE,31,gen,...,False,PETE AGUILAR,DEMOCRAT,False,TOTAL,51622,99784,False,20201202,False
25685,True,2014,FLORIDA,FL,12,59,43,US HOUSE,2,gen,...,False,GWEN GRAHAM,DEMOCRAT,False,TOTAL,126096,249780,False,20201202,False
25745,True,2014,FLORIDA,FL,12,59,43,US HOUSE,26,gen,...,False,CARLOS CURBELO,REPUBLICAN,False,TOTAL,83031,161337,False,20201202,False
25766,True,2014,GEORGIA,GA,13,58,44,US HOUSE,12,gen,...,False,"R. W. """"RICK"""" ALLEN",REPUBLICAN,False,TOTAL,91336,166814,False,20201202,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29486,True,2018,VIRGINIA,VA,51,54,40,US HOUSE,7,gen,...,False,ABIGAIL A. SPANBERGER,DEMOCRAT,False,TOTAL,176079,349803,False,20201202,False
29496,True,2018,VIRGINIA,VA,51,54,40,US HOUSE,10,gen,...,False,JENNIFER T. WEXTON,DEMOCRAT,False,TOTAL,206356,367795,False,20201202,False
29517,True,2018,WASHINGTON,WA,53,91,73,US HOUSE,8,gen,...,False,KIM SCHRIER,DEMOCRAT,False,TOTAL,164089,313057,False,20201202,False
29591,True,2018,GEORGIA,GA,13,58,44,US HOUSE,6,gen,...,False,LUCY MCBATH,DEMOCRAT,False,TOTAL,160139,317014,False,20201202,False


In [18]:
res.loc[(res["state_fips"] == 23) & (res["district"] == 2)]

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,fusion_ticket
421,1976,MAINE,ME,23,11,2,US HOUSE,2,gen,False,False,LEIGHTON COONEY,DEMOCRAT,False,TOTAL,43150,219547,False,20201202,False
422,1976,MAINE,ME,23,11,2,US HOUSE,2,gen,False,False,WILLIAM S. COHEN,REPUBLICAN,False,TOTAL,169292,219547,False,20201202,False
423,1976,MAINE,ME,23,11,2,US HOUSE,2,gen,False,False,JACQUELINE FRANCIS KAYE,INDEPENDENT,False,TOTAL,7105,219547,False,20201202,False
1686,1978,MAINE,ME,23,11,2,US HOUSE,2,gen,False,False,OLYMPIA J. SNOWE,REPUBLICAN,False,TOTAL,87939,173037,False,20201202,False
1687,1978,MAINE,ME,23,11,2,US HOUSE,2,gen,False,False,MARGARET E. COUSINS,INDEPENDENT,False,TOTAL,1573,173037,False,20201202,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27292,2016,MAINE,ME,23,11,2,US HOUSE,2,gen,NaN,False,EMILY ANN CAIN,DEMOCRAT,False,TOTAL,159081,364886,False,20201202,False
27293,2016,MAINE,ME,23,11,2,US HOUSE,2,gen,NaN,False,BRUCE POLIQUIN,REPUBLICAN,False,TOTAL,192878,364886,False,20201202,False
27294,2016,MAINE,ME,23,11,2,US HOUSE,2,gen,NaN,False,NaN,NaN,True,TOTAL,224,364886,False,20201202,False
29599,2018,MAINE,ME,23,11,2,US HOUSE,2,gen,NaN,False,JARED F. GOLDEN,DEMOCRAT,False,TOTAL,139231,281371,False,20201202,False


### Demographics

In [21]:
import requests
import urllib.request

#download the data and put into list of lists
def data(year, columns):
    url = "https://api.census.gov/data/"+str(year)+"/acs/acs5/subject?get=NAME,"+ ",".join(columns)+"&for=congressional%20district:*"
    file = urllib.request.urlopen(url)

    x = []
    for line in file:
        decoded_line = line.decode("utf-8").split(",")
        for i in range(len(decoded_line)):
            try: #A little bit of preproccesing
                decoded_line[i] = decoded_line[i].strip("\"").strip("\"]").strip("[\"")
                if decoded_line[i] == -666666666:
                    decoded_line[i] = 0
            except:
                pass
        #remove ugly header
        decoded_line.pop()
        x.append(decoded_line)
    x.pop(0)
    return x

In [22]:
#from census data
columns_1516171819 = ["S2301_C02_012E","S2301_C02_013E","S2301_C02_014E","S2301_C02_015E","S2301_C02_016E","S2301_C02_017E","S2301_C02_018E"]
columns_1011121314 = ["S2301_C02_010E","S2301_C02_011E","S2301_C02_012E","S2301_C02_013E","S2301_C02_014E","S2301_C02_015E","S2301_C02_016E"]

columns = ["name","state","White","Black/AfricanAmerican","AmericanIndian/AlaskanNative","Asian",
           "NativeHawaiian/PacificIslander", "Other", "TwoOrMore", "state_code","district"]

#create a dataframe for each year.
dems_18 = pd.DataFrame(data(2018,columns_1516171819), columns=columns)

dems_16 = pd.DataFrame(data(2016,columns_1516171819), columns=columns)

dems_14 = pd.DataFrame(data(2014,columns_1011121314), columns=columns)

dems_12 = pd.DataFrame(data(2012,columns_1011121314), columns=columns)

dems_10 = pd.DataFrame(data(2010,columns_1011121314), columns=columns)

https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S2301_C02_012E,S2301_C02_013E,S2301_C02_014E,S2301_C02_015E,S2301_C02_016E,S2301_C02_017E,S2301_C02_018E&for=congressional%20district:*
https://api.census.gov/data/2018/acs/acs5/subject?get=NAME,S2301_C02_012E,S2301_C02_013E,S2301_C02_014E,S2301_C02_015E,S2301_C02_016E,S2301_C02_017E,S2301_C02_018E&for=congressional%20district:*
https://api.census.gov/data/2016/acs/acs5/subject?get=NAME,S2301_C02_012E,S2301_C02_013E,S2301_C02_014E,S2301_C02_015E,S2301_C02_016E,S2301_C02_017E,S2301_C02_018E&for=congressional%20district:*
https://api.census.gov/data/2014/acs/acs5/subject?get=NAME,S2301_C02_010E,S2301_C02_011E,S2301_C02_012E,S2301_C02_013E,S2301_C02_014E,S2301_C02_015E,S2301_C02_016E&for=congressional%20district:*
https://api.census.gov/data/2012/acs/acs5/subject?get=NAME,S2301_C02_010E,S2301_C02_011E,S2301_C02_012E,S2301_C02_013E,S2301_C02_014E,S2301_C02_015E,S2301_C02_016E&for=congressional%20district:*
https://api.census.gov/da

In [23]:
#preproccesing
def types_transf(df):
    df["White"] = df["White"].astype(float)
    df["Black/AfricanAmerican"] = df["Black/AfricanAmerican"].astype(float)
    df["AmericanIndian/AlaskanNative"] = df["AmericanIndian/AlaskanNative"].astype(float)
    df["Asian"] = df["Asian"].astype(float)
    df["NativeHawaiian/PacificIslander"] = df["NativeHawaiian/PacificIslander"].astype(float)
    df["Other"] = df["Other"].astype(float)
    df["TwoOrMore"] = df["TwoOrMore"].astype(float)
    df = df.replace(-666666666.0, 0)
    return df

In [24]:
#apply preproccesing
dems_18t = types_transf(dems_18)
dems_16t = types_transf(dems_16)
dems_14t = types_transf(dems_14)
dems_12t = types_transf(dems_12)
dems_10t = types_transf(dems_10)

In [25]:
#one way to find difference. Make one table negative, apppend to other table, then group by and aggregate sum
def neg(df):
    df["White"] = -df["White"]
    df["Black/AfricanAmerican"] = -df["Black/AfricanAmerican"]
    df["AmericanIndian/AlaskanNative"] = -df["AmericanIndian/AlaskanNative"]
    df["Asian"] = -df["Asian"]
    df["NativeHawaiian/PacificIslander"] = -df["NativeHawaiian/PacificIslander"]
    df["Other"] = -df["Other"]
    df["TwoOrMore"] = -df["TwoOrMore"]
    return df

In [26]:
dem_18dif = dems_18t.append(neg(dems_16t))

In [27]:
dems_18t.to_csv("demographics2018.csv", index = False)
dems_16t.to_csv("demographics2016.csv", index = False)
dems_14t.to_csv("demographics2014.csv", index = False)
dems_12t.to_csv("demographics2012.csv", index = False)
dems_10t.to_csv("demographics2010.csv", index = False)

In [28]:
dems_18t

,name,state,White,Black/AfricanAmerican,AmericanIndian/AlaskanNative,Asian,NativeHawaiian/PacificIslander,Other,TwoOrMore,state_code,district
0,Congressional District 1 (116th Congress),Alabama,56.7,53.8,48.4,66.1,95.5,72.6,56.9,01,01
1,Congressional District 3 (116th Congress),Alabama,56.6,57.4,54.7,62.6,47.4,63.9,58.1,01,03
2,Congressional District 5 (116th Congress),Alabama,58.8,63.7,61.7,64.4,66.4,65.6,67.4,01,05
3,Congressional District 4 (116th Congress),Alabama,53.9,53.9,55.7,63.6,74.2,69.3,45.7,01,04
4,Congressional District 7 (116th Congress),Alabama,55.7,54.5,50.6,62.0,50.2,69.0,60.3,01,07
...,...,...,...,...,...,...,...,...,...,...,...
435,Congressional District 7 (116th Congress),New York,68.8,56.5,63.8,60.7,57.0,60.2,69.1,36,07
436,Congressional District 17 (116th Congress),New York,63.8,65.7,78.7,67.5,64.4,78.1,71.0,36,17
437,Congressional District 6 (116th Congress),New York,60.1,66.1,69.1,61.9,70.9,70.3,65.9,36,06
438,Congressional District 5 (116th Congress),New York,56.9,66.1,57.4,63.6,64.2,70.3,63.6,36,05
